## pyTopoComplexity (v0.5.4)
### **Landform Smoothing via Nonlinear Hillslope Diffusion Modeling**

This notebook demonstrates the use of **Landlab** (Hobley et al., 2017), a open-source Python framework for simulating landscape evolution, to model topographic smoothing driven by near-surface soil disturbance and downslope soil creep processes. Specifically, this notebook employs the `TaylorNonLinearDiffuser` component from LandLab, described as one element in the **terrainBento** package (Barnhart et al., 2019), to simulate topographic smoothing over time through non-linear hillslope diffusion processes (Roering et al., 1999).

The example GeoTIFF rasters include the LiDAR Digital Elevation Model (DEM) files that cover the area and nearby region of a deep-seated landslide occurred in 2014 at Oso area of the North Fork Stillaguamish River (NFSR) valley, Washington State, USA (Washington Geological Survey, 2023). The example DEMs have various grid size, coordinate reference system (CRS), and unit of grid value (elevation, Z). 

**WARNING**: There is a known/unresolved stability issue when running `TaylorNonLinearDiffuser` component with a DEM with reprojected coordinate reference system (CRS) through GIS softwares. When using the example DEM, users may only use the original DEM with CRS: NAD83/Washington South (ftUS) (EPSG: 2286) and Z unit in US survey feet (e.g., the DEM files named with *"_f_3ftgrid"* or *"_f_6ftgrid"*).

To use this code, please cite the Zenodo repository that hosts the latest release of this code: 
* Lai, L. S.-H. (2024). pyTopoComplexity. Zenodo. https://doi.org/10.5281/zenodo.11239338
* Github repository: https://github.com/LarrySHLai/pyTopoComlexity

<hr>
<small>For more about Landlab installation and tutorials, visit: <a href="https://landlab.readthedocs.io/en/latest/index.html">https://landlab.readthedocs.io/en/latest/index.html</a></small>
<hr>

### **Theory** (adapted from **Landlab**'s tutorials)
Consider a topographic surface where the elevation at any time $t$ and horizontal position $(x,y)$ is $z(x,y,t)$. Let $\mathbf{q}_s$ be a 2D vector representing the rate of soil volume flow per unit slope width (with dimensions of length squared per time; we'll assume that $\mathbf{q}_s$ represents a "bulk" flux that includes pore spaces between soil grains).
In the absence of any "local" input sources (such as weathering of rock) or output (such as removal by wash erosion), conservation of mass dictates that:

$$\frac{\partial z}{\partial t} = -\nabla \cdot \mathbf{q}_s$$

The soil flux ($\mathbf{q}_s$) can be represented as:

$$\mathbf{q}_s = D \mathbf{S} \left[1 + \sum_{i=1}^N \left( \frac{S}{S_c}\right)^{2i}\right]$$

, where $\mathbf{S} = -\nabla z$ is the downslope topographic gradient, and $S$ is its magnitude. Parameter $K$ is a diffusion-like transport coefficient with dimensions of length squared per time. The $i$ is the number of additional terms desired. If $i=0$, the expression reduces to plain linear diffusion.

The simulation takes into account the critical slope gradient ($S_c$) and uses a Taylor expansion to approximate the nonlinear diffusion process. Inspired by Ganti et al. (2012), the `TaylorNonLinearDiffuser` component uses a slope-dependent flux law with a user-specified number of terms in a Taylor expansion. This approximates the Andrews-Bucknam transport function (e.g., Roering et al., 1999) while avoiding that equation's blow-up at $S=S_c$.
<hr>

### **References**
##### Journal Articles: 
* Barnhart, K., Glade, R., Shobe, C., Tucker, G. (2019). Terrainbento 1.0: a Python package for multi-model analysis in long-term drainage basin evolution. Geoscientific Model Development  12(4), 1267-1297. https://doi.org/10.5194/gmd-12-1267-2019
* Booth, A.M., LaHusen, S.R., Duvall, A.R., Montgomery, D.R., 2017. Holocene history of deep-seated landsliding in the North Fork Stillaguamish River valley from surface roughness analysis, radiocarbon dating, and numerical landscape evolution modeling. Journal of Geophysical Research: Earth Surface 122, 456-472. https://doi.org/10.1002/2016JF003934  
* Ganti, V., Passalacqua, P., Foufoula-Georgiou, E. (2012). A sub-grid scale closure for nonlinear hillslope sediment transport models. Journal of Geophysical Research: Earth Surface, 117(F2). https://doi.org/10.1029/2011jf002181
* Hobley, D.E.J., Adams, J.M., Nudurupati, S.S., Hutton, E.W.H., Gasparini, N.M., Istanbulluoglu, E., Tucker, G.E., 2017. Creative computing with Landlab: an open-source toolkit for building, coupling, and exploring two-dimensional numerical models of Earth-surface dynamics. Earth Surf. Dynam. 5, 21-46. https://doi.org/10.5194/esurf-5-21-2017
* Roering, J. J., Kirchner, J. W., & Dietrich, W. E. (1999). Evidence for nonlinear, diffusive sediment transport on hillslopes and implications for landscape morphology. Water Resources Research, 35(3), 853-870. https://doi.org/10.1029/1998wr900090

##### Digital Elevation Model (DEM) Examples:
* Washington Geological Survey, 2023. 'Stillaguamish 2014' and 'Snohoco Hazel 2006' projects [lidar data]: originally contracted by Washington State Department of Transportation (WSDOT). [accessed April 4, 2024, at http://lidarportal.dnr.wa.gov]

<hr>

#### 0. Import Libraries and Set Constants
This section imports necessary libraries and sets up constants for file paths.

In [ ]:
import os
import numpy as np
import rasterio
from osgeo import gdal
import matplotlib.pyplot as plt
from landlab import imshowhs_grid
from landlab.components import TaylorNonLinearDiffuser
from landlab.io import read_esri_ascii, write_esri_ascii

#### 1. Define File Directories
This section defines file directories for input files and simulation results.
The default assumes the input and output GeoTIFF rasters will be placed in the same directory - a subfolder named <code>'ExampleDEM'</code>

In [ ]:
BASE_DIR = os.path.join(os.getcwd(), 'ExampleDEM')
INPUT_TIFF = 'Ososlid2014_f_6ftgrid.tif'
OUT_DIR = os.path.join(BASE_DIR, 'simulation_results')
OUT_DIRpng = os.path.join(OUT_DIR, 'PNGs')
OUT_DIRtiff = os.path.join(OUT_DIR, 'GeoTIFFs')
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(OUT_DIRpng, exist_ok=True)
os.makedirs(OUT_DIRtiff, exist_ok=True)

#### 2. Utility Functions
These utility functions handle file format conversions, because **Landlab** are designed for processing raster files in ESRI ASCII format rather than GeoTIFFs.
* `tiff_to_asc`: Converts GeoTIFF to ESRI ASCII format.
* `asc_to_tiff`: Converts ESRI ASCII back to GeoTIFF, preserving metadata.

***Notes***: The input DEM layout must not have nodata or extreme values to prevent `TaylorNonLinearDiffuser` component encountering errors due to infinite slope ($S$) and sediment flux ($\mathbf{q}_s$).

In [ ]:
def tiff_to_asc(in_path, out_path):
    with rasterio.open(in_path) as src:
        XYZunit = src.crs.linear_units   #assuming the unit of XYZ direction are the same
        mean_res = np.mean(src.res)
    
    gdal.Translate(out_path, in_path, format='AAIGrid', xRes=mean_res, yRes=mean_res)
    print(f"The input GeoTIFF is temporarily converted to '{os.path.basename(out_path)}' with grid spacing {mean_res} ({XYZunit})")
    return mean_res, XYZunit
    
def asc_to_tiff(asc_path, tiff_path, meta):
    data = np.loadtxt(asc_path, skiprows=6)
    meta.update(dtype=rasterio.float32, count=1, compress='deflate')

    with rasterio.open(tiff_path, 'w', **meta) as dst:
        dst.write(data.astype(rasterio.float32), 1)
    print(f"'{os.path.basename(tiff_path)}' saved to 'simulation_results' folder")

#### 3. Function for Simulation Initialization
This function initializes the simulation:
1. Reads the ASCII file into a Landlab grid.
2. Sets boundary conditions.
3. Converts the diffusion coefficient to appropriate units.
4. Creates and returns the `TaylorNonLinearDiffuser` component.

In [ ]:
ft2mUS = 1200/3937 #US survey foot to meter conversion factor 
ft2mInt = 0.3048 #International foot to meter conversion factor 

def init_simulation(asc_file, K, Sc, XYZunit=None):
    grid, z = read_esri_ascii(asc_file, name='topographic__elevation') #the xy grid spacing must be equal
    grid.set_closed_boundaries_at_grid_edges(False, False, False, False)

    # Check the unit of XYZ and make unit conversion of K when needed
    if XYZunit is None:
        print("The function assumes the input XYZ units are in meters.")
        TNLD = TaylorNonLinearDiffuser(grid, linear_diffusivity=K, slope_crit=Sc, dynamic_dt=True, nterms=2, if_unstable = "pass")
    elif XYZunit is not None:
        if any(unit in XYZunit.lower() for unit in ["metre".lower(), "meter".lower()]):
            print("Input XYZ units are in meters. No unit conversion is made")
            TNLD = TaylorNonLinearDiffuser(grid, linear_diffusivity=K, slope_crit=Sc, dynamic_dt=True, nterms=2, if_unstable = "pass")
        elif any(unit in XYZunit.lower() for unit in ["foot".lower(), "feet".lower(), "ft".lower()]):  
            if any(unit in XYZunit.lower() for unit in ["US".lower(), "United States".lower()]):
                print("Input XYZ units are in US survey feet. A unit conversion to meters is made for K")
                Kc = K / (ft2mUS ** 2)
                TNLD = TaylorNonLinearDiffuser(grid, linear_diffusivity=Kc, slope_crit=Sc, dynamic_dt=True, nterms=2, if_unstable = "pass")
            else:
                print("Input XYZ units are in international feet. A unit conversion to meters is made for K")
                Kc = K / (ft2mInt ** 2)
                TNLD = TaylorNonLinearDiffuser(grid, linear_diffusivity=Kc, slope_crit=Sc, dynamic_dt=True, nterms=2, if_unstable = "pass")
        else:
            message = (
            "WARNING: The code excution is stopped. "
            "The input XYZ units must be in feet or meters."
            )
            raise RuntimeError(message)

    return grid, TNLD

#### 4. Function for Plotting and Saving Results
This function handles visualization and data saving:
1. Creates a hillshade plot of the current topography.
2. Saves the plot as a PNG file.
3. Saves the elevation data as a temporary ASCII file.

In [ ]:
def plot_save(grid, z, basefilename, time, K, mean_res=None, XYZunit=None):
    plt.figure(figsize=(6, 5.25))
    imshowhs_grid(grid, z, plot_type="Hillshade")
    plt.title(f"{basefilename} \n Time: {time} years (K = {K} m$^{{2}}$/yr)", fontsize='small', fontweight="bold")
    plt.xticks(fontsize='small')
    plt.yticks(fontsize='small')
    if XYZunit is not None:
        plt.xlabel(f'X-axis grids \n(grid size ≈ {round(mean_res, 4)} [{XYZunit}])', fontsize='small')
        plt.ylabel(f'Y-axis grids \n(grid size ≈ {round(mean_res, 4)} [{XYZunit}])', fontsize='small')
    else:
        print("The function assumes the input XYZ units are in meters.")
        plt.xlabel(f'X-axis grids \n(grid size ≈ {1.0} [meters])', fontsize = 'small')
        plt.ylabel(f'Y-axis grids \n(grid size ≈ {1.0} [meters])', fontsize = 'small') 
    plt.tight_layout()
    #plt.show()
    plt.savefig(os.path.join(OUT_DIRpng, f"{basefilename}_{time}yrs_(K={K}).png"), dpi=150)
    plt.close()
    print(f"'{basefilename}_{time}yrs_(K={K}).png' saved to 'simulation_results' folder")

    asc_path = os.path.join(OUT_DIRtiff, f"{basefilename}_{time}_(K={K})yrs.asc")
    write_esri_ascii(asc_path, grid, names=['topographic__elevation'], clobber=True)
    
    return asc_path

#### 5. Main Simulation Function
This is the main function that runs the simulation:
1. Converts input TIFF to ASCII.
2. Initializes the simulation.
3. Runs the diffusion model for the specified number of time steps.
4. For each step, it updates the topography, saves results, and converts files.
5. Cleans up temporary files (ASCII and PRJ files) at the end.

In [ ]:
def run_simulation(in_tiff, K, Sc, dt, target_time):
    basefilename = os.path.splitext(in_tiff)[0]
    in_asc = os.path.join(BASE_DIR, f"{basefilename}.asc")
    mean_res, XYZunit = tiff_to_asc(os.path.join(BASE_DIR, in_tiff), in_asc) #convert input GeoTIFF to ASCII, and determine the XYZ units

    grid, tnld = init_simulation(in_asc, K, Sc, XYZunit)
    z = grid.at_node['topographic__elevation']

    with rasterio.open(os.path.join(BASE_DIR, in_tiff)) as src:
        meta = src.meta.copy()

    asc_path = plot_save(grid, z, basefilename, 0, K, mean_res, XYZunit)
    os.remove(asc_path)

    num_steps = int(target_time / dt)
    for i in range(num_steps):
        tnld.run_one_step(dt)
        time = (i + 1) * dt
        asc_path = plot_save(grid, z, basefilename, time, K, mean_res, XYZunit)
        
        tiff_path = os.path.join(OUT_DIRtiff, f"{basefilename}_{time}yrs_(K={K}).tif")
        asc_to_tiff(asc_path, tiff_path, meta)
        
        os.remove(asc_path)
        
    in_prj = in_asc.replace('.asc', '.prj')
    os.remove(in_asc)
    os.remove(in_prj)
    print("Simulation completed. Temporary ASCII & PRJ files cleaned up.")

#### 6. Excuting the Simulation
This section sets the simulation parameters and runs the simulation when the script is executed directly.

When we include nonlinear terms in the transport law, we expect to see slopes that become more planar in character. Here the critical slope value is set to $S_c = 1.25$ (about $51.34^\circ$), the average slope of the Oso landslide area in the Washington State, USA. The code has two terms in the Taylor expansion (`ntems = 2`), one linear and one cubic, that gives the behavior described in Ganti et al. (2012) as an approximation of the nonlinear diffusion. The code also invokes the `dynamic_dt` option, which allows the component to subdivide each "global" timestep if needed for numerical stability.

In [ ]:
if __name__ == "__main__":
    K = 0.0029  # diffusion coefficient, m^2/y
    Sc = 1.25   # critical slope gradient, m/m
    dt = 1000   # time step size (years)
    end_time = 15000  # final simulation time (years)

    run_simulation(INPUT_TIFF, K, Sc, dt, end_time)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Image

def display_image(time):
    filename = f"{os.path.splitext(INPUT_TIFF)[0]}_{time}yrs_(K={K}).png"
    filepath = os.path.join(OUT_DIRpng, filename)
    if os.path.exists(filepath):
        display(Image(filename=filepath))
    else:
        print("No file found for this year.")

print(f"Diffusion coefficient K = {K} m²/yr")

# Create a slider of time (dt)
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=end_time,  # end_time should be defined in your earlier cells
    step=dt,  # dt should be defined as your timestep size in years
    description='Time (years):', #Name of the slider
    continuous_update=False
)

# Interactive display of the image
widgets.interactive(display_image, time=slider)